In [1]:
import argparse
from math import sqrt
from random import choice, sample

import community as louvain
import networkx as nx
from tqdm import tqdm

from communityDetection import load_network
from evaluation import calculate_nmi, load_ground_truth
from models import *
from models.nlta import frequentest_label, max_freq_check
from utils import community_to_node, eval

In [2]:
args = argparse.Namespace(
    networkFile='TC2/tc11.dat',
    communityFile=None,
    communitySavePath='TC2/tc11.cmty',
)

graph = load_network(args.networkFile)

In [3]:
from models import my_algorithm

my_algorithm(graph, 0.3, 2, None, epochs=50)

Epoch       0:  99%|█████████▉| 9885/10000 [00:01<00:00, 7546.25it/s]

TypeError: expected str, bytes or os.PathLike object, not NoneType

In [ ]:
def get_relationship(node1, node2, graph: nx.classes.graph.Graph):
    neighbors_1 = set(graph.neighbors(node1))
    neighbors_2 = set(graph.neighbors(node2)) | {node2}

    relationship = len(neighbors_1 & neighbors_2) / len(neighbors_1)
    return relationship


def get_relative_neighbors(node1, epsilon, graph: nx.classes.graph.Graph):
    neighbors_1 = graph.neighbors(node1)
    result = tuple(node for node in neighbors_1 if get_relationship(node1, node, graph) >= epsilon)

    return result


# def binary_to_decimal(binary: tuple[float]):
#     result = sum(2**exponent * coef for exponent, coef in enumerate(binary[::-1]))

#     return result


# def binary_mean(binary1: tuple[float], *args):
#     result = tuple(sum(coefs)/len(coefs) for coefs in zip(binary1, *args))

#     return result

In [ ]:
def my_nlta(graph: nx.classes.graph.Graph, epsilon, true_file_path=None, max_epochs=100):
    nodes = tuple(graph.nodes())
    nodes_num = len(nodes)
    stop_iteration = False
    epoch = 0

    indices = sample(range(nodes_num), k=nodes_num)
    nodes = tuple(nodes[index] for index in indices)
    del indices
    labels = dict((node, node) for node in nodes)

    while stop_iteration is False and epoch < max_epochs:
        # phase 1
        pbar = tqdm(total=nodes_num, desc=f'Epoch {epoch:3d}')
        pbar_update = pbar.update
        postfix = dict()
        non_neighbor_count = 0

        for node in nodes:
            neighbors = get_relative_neighbors(node, epsilon, graph)
            neighbors_size = len(neighbors)
            if neighbors_size == 0:
                neighbors = tuple(graph.neighbors(node))
                neighbors_size = len(neighbors)
                non_neighbor_count += 1

            neighbors_labels = tuple(labels[neighbor] for neighbor in neighbors)
            # labels[node] = choice(neighbors_labels)
            labels[node] = frequentest_label(neighbors_labels)

            pbar_update()
            
        if true_file_path is not None:
            eval_score = eval(labels, true_file_path)
            postfix['eval score'] = f'{eval_score:6.1%}'
            
        postfix['no friend'] = f'{non_neighbor_count:4d}'
        pbar.set_postfix(postfix)
        pbar.close()

        # phase 2
        for node in nodes:
            check = max_freq_check(node, labels[node], labels, graph)
            if check is False:
                stop_iteration = False
                break
            
            stop_iteration = True

        # etc.
        epoch += 1

    labels = dict((key, (round(value) if value is not None else None)) for key, value in labels.items())
    labels = dict(sorted(labels.items(), key=lambda x: x[0]))  # sort by node in ascending order

    return labels

In [ ]:
labels = nlta_algorithm(graph, args.communityFile, 10)